# Qcodes example with Swabian Instruments Time Tagger

In [1]:
from qcodes_contrib_drivers.drivers.SwabianInstruments.Swabian_Instruments_Time_Tagger import TimeTagger

The driver wraps the `TimeTagger` python API provided by Swabian Instruments (download [here](https://www.swabianinstruments.com/time-tagger/downloads/)). As the API is very object-oriented, the QCoDeS driver is fairly dynamic in that, for each new measurement type, a new `InstrumentChannel` of the corresponding type is added to the main instrument. These channels have parameters which correspond to the instantiation arguments of the API objects, as well as gettable parameters which correspond to the measured data.

For more information on the driver design, see the `SwabianInstruments/Swabian_Instruments_Time_Tagger.py` module docstring.

In [2]:
tagger = TimeTagger('tagger')

Connected to: Swabian Instruments Time Tagger 20 (serial:12345678XX, firmware:TT-20, FW4, OK 3.1) in 1.38s


To add a new `Correlation` measurement, use the automatically generated helper method `add_correlation_measurement`, which adds a `CorrelationMeasurement` channel to the `correlation_measurements` channel list.

In [3]:
correlation = tagger.add_correlation_measurement()
correlation

<CorrelationMeasurement: tagger_correlation_1 of TimeTagger: tagger>

At first, the API object is not yet instantiated because arguments are missing:

In [4]:
try:
    correlation.api
except RuntimeError as err:
    print(err)

The following parameters need to be initialized first: channels


Instead, we must provide the arguments by setting the parameters (note that units are the same as the API driver, so usually picoseconds):

In [5]:
correlation.channels([1, 2])
correlation.binwidth(100)
correlation.n_bins(1000)
correlation.api

<TimeTagger.Correlation; proxy of <Swig Object of type 'Correlation *' at 0x000001DB44D97000> >

We can now start measuring:

In [6]:
correlation.start_for(5 * 10 ** 12, clear=True)
while correlation.is_running():
    pass
print(correlation.capture_duration())
correlation.data_normalized()

5000000000000


array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0.

Once a measurement is not needed anymore, it may also be removed from the tagger object:

In [7]:
tagger.correlation_measurements.remove(correlation)

## Synchronized measurements

The `TimeTagger` driver also implements the `SynchronizedMeasurements` functionality of the API. This is an object that helps to synchronize multiple different measurements using the same physical tagger. It can either be used by passing the virtual tagger return by its `get_tagger()` method to the measurement, or by calling the `register_measurement()` and `unregister_measurement()` methods:

In [8]:
tagger.synchronized_measurements

<TimeTaggerSynchronizedMeasurements: tagger_synchronized_measurements of TimeTagger: tagger>

In [9]:
sync_tagger = tagger.synchronized_measurements.api_tagger
sync_correlation = tagger.add_correlation_measurement(api_tagger=sync_tagger)
sync_count_rate = tagger.add_count_rate_measurement(api_tagger=sync_tagger)
# Perform measurement as usual

Or, equivalently (note that parameters need to be initialized):

In [10]:
correlation = tagger.add_correlation_measurement()
correlation.channels([-1, -2])  # falling flank

count_rate = tagger.add_count_rate_measurement()
count_rate.channels([-1, -2])

tagger.synchronized_measurements.register_measurement(correlation)
tagger.synchronized_measurements.register_measurement(count_rate)
# Perform measurement as usual
tagger.synchronized_measurements.unregister_measurement(correlation)
tagger.synchronized_measurements.unregister_measurement(count_rate)

In [11]:
tagger.close()